<a href="https://colab.research.google.com/github/izk8/2022_siop_fri_seminar/blob/main/22_SIOP_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load relivant packages
!pip install -U sentence-transformers
!pip install ipython-autotime

In [2]:
# bring in relivant models
# explain models and model choice
# print(pd.__version__)
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import pickle
%load_ext autotime 
import time

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

time: 4.34 s (started: 2022-04-12 18:42:42 +00:00)


In [3]:
job_dict = pd.read_pickle(r'/content/job_dict.pkl')

time: 85.1 ms (started: 2022-04-12 18:42:47 +00:00)


In [23]:
job_dict["df"]

0       Role: To assist members and potential members ...
1       JOB FUNCTION / SUMMARY: A Multi-Service Banker...
2       Job Summary At Blue Sky, we recognize the need...
3       JOB DESCRIPTION Position: Store Manager Report...
4       ALL APPLICATIONS MUST BE SUBMITTED ONLINE AT h...
                              ...                        
9995    Position Summary To perform an expanded clinic...
9996    The Case Manager is responsible for complete c...
9997    Job Title: Property Accountability & Materials...
9998    The Mary S. Shook Student Health Service exist...
9999    Our mission at the State of Connecticut, Depar...
Name: job_description, Length: 10000, dtype: object

time: 5.72 ms (started: 2022-04-12 18:54:37 +00:00)


In [5]:
job_titles = job_dict["df"]["job_title"]

time: 2.56 ms (started: 2022-04-12 18:42:47 +00:00)


In [6]:
job_titles

0                               Member Service Specialist
1                           Floating Multi Service Banker
2                           Sales Associate Blue Sky #711
3                                           Store Manager
4                                     Planning Technician
                              ...                        
9995    Nurse Practitioner - Dermatology (Mon-Fri) OUT...
9996               Substance Abuse Treatment Case Manager
9997                                    Supply Technician
9998    Temporary PRN Nurse Practitioner/Physician Ass...
9999           Clinical Social Worker Associate (35 Hour)
Name: job_title, Length: 10000, dtype: object

time: 3.97 ms (started: 2022-04-12 18:42:47 +00:00)


# New Section

In [7]:
# make embeddings of titles
title_emb = model.encode(job_titles, show_progress_bar = True, convert_to_tensor=True)
#without gpu and 100k this will take 30 min +, change runtime to gpu... will take 30 sec... 

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

time: 12.5 s (started: 2022-04-12 18:42:47 +00:00)


In [8]:
title_emb.shape

torch.Size([10000, 384])

time: 2.93 ms (started: 2022-04-12 18:42:59 +00:00)


In [9]:
title_emb

tensor([[-0.4357, -0.4224, -0.2424,  ..., -0.3637,  0.1671,  0.4540],
        [-0.3686, -0.6254, -0.7172,  ..., -0.4637,  0.3951, -0.2661],
        [-0.2146, -0.4029,  0.1744,  ..., -0.7837, -0.7946,  0.3658],
        ...,
        [-0.9767, -0.3604, -0.1257,  ..., -0.7109,  0.7241,  0.0203],
        [-0.5571, -0.1584,  0.0423,  ..., -0.0864, -0.2682,  0.5536],
        [ 0.0519, -0.0374,  0.4928,  ..., -0.3209, -0.0911,  0.0488]],
       device='cuda:0')

time: 26.2 ms (started: 2022-04-12 18:42:59 +00:00)


In [10]:
cosine_scores = util.cos_sim(title_emb, title_emb)
cosine_scores

tensor([[1.0000, 0.3860, 0.3343,  ..., 0.4596, 0.3913, 0.4455],
        [0.3860, 1.0000, 0.2354,  ..., 0.3855, 0.2120, 0.1656],
        [0.3343, 0.2354, 1.0000,  ..., 0.2886, 0.1167, 0.3075],
        ...,
        [0.4596, 0.3855, 0.2886,  ..., 1.0000, 0.3372, 0.3713],
        [0.3913, 0.2120, 0.1167,  ..., 0.3372, 1.0000, 0.4632],
        [0.4455, 0.1656, 0.3075,  ..., 0.3713, 0.4632, 1.0000]],
       device='cuda:0')

time: 18.5 ms (started: 2022-04-12 18:43:17 +00:00)


In [ ]:
# #Find the pairs with the highest cosine similarity scores
# pairs = []
# for i in range(len(cosine_scores)-1):
#     for j in range(i+1, len(cosine_scores)):
#         pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})


In [ ]:
#Sort scores in decreasing order
#pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

time: 39.3 s (started: 2022-03-10 21:18:18 +00:00)


In [ ]:
# for pair in pairs[0:10]:
#     i, j = pair['index']
#     print("{} \t\t {} \t\t Score: {:.4f}".format(title_emb[i], title_emb[j], pair['score']))

recruiter 		 recruiter 		 Score: 1.0000
recruiter 		 recruiter 		 Score: 1.0000
recruiter 		 recruiter 		 Score: 1.0000
recruiter 		 recruiter 		 Score: 1.0000
recruiter 		 recruiter 		 Score: 1.0000
recruiter 		 recruiter 		 Score: 1.0000
pastry cook 		 pastry cook 		 Score: 1.0000
front desk receptionist 		 front desk receptionist 		 Score: 1.0000
retail sales associate 		 retail sales associate 		 Score: 1.0000
retail sales associate 		 retail sales associate 		 Score: 1.0000
time: 14.4 ms (started: 2022-03-10 21:18:58 +00:00)


In [11]:
input_job_title = "Jr. Java Script Developer"
input_embedding = model.encode(input_job_title, convert_to_tensor=True)

time: 15 ms (started: 2022-04-12 18:43:45 +00:00)


In [12]:
results = util.semantic_search(input_embedding, title_emb, top_k = 10)

time: 7.89 ms (started: 2022-04-12 18:43:48 +00:00)


In [13]:
results

[[{'corpus_id': 8126, 'score': 0.8811661005020142},
  {'corpus_id': 3703, 'score': 0.8493909239768982},
  {'corpus_id': 2320, 'score': 0.8493907451629639},
  {'corpus_id': 9425, 'score': 0.8493907451629639},
  {'corpus_id': 8949, 'score': 0.8305301666259766},
  {'corpus_id': 3717, 'score': 0.8197875618934631},
  {'corpus_id': 418, 'score': 0.7895177006721497},
  {'corpus_id': 4225, 'score': 0.785468578338623},
  {'corpus_id': 8147, 'score': 0.7689410448074341},
  {'corpus_id': 88, 'score': 0.7544420957565308}]]

time: 4.03 ms (started: 2022-04-12 18:44:00 +00:00)


In [ ]:
#results
#results[0][0] # first one is first seach, second one is the first dictionary... 
#results[0][0]['corpus_id'] #gets index

{'corpus_id': 71322, 'score': 0.9147287011146545}

time: 3.17 ms (started: 2022-03-10 20:45:35 +00:00)


In [16]:
results

[[{'corpus_id': 8126, 'score': 0.8811661005020142},
  {'corpus_id': 3703, 'score': 0.8493909239768982},
  {'corpus_id': 2320, 'score': 0.8493907451629639},
  {'corpus_id': 9425, 'score': 0.8493907451629639},
  {'corpus_id': 8949, 'score': 0.8305301666259766},
  {'corpus_id': 3717, 'score': 0.8197875618934631},
  {'corpus_id': 418, 'score': 0.7895177006721497},
  {'corpus_id': 4225, 'score': 0.785468578338623},
  {'corpus_id': 8147, 'score': 0.7689410448074341},
  {'corpus_id': 88, 'score': 0.7544420957565308}]]

time: 3.26 ms (started: 2022-04-12 18:45:22 +00:00)


In [19]:
results[0][0]['corpus_id']

8126

time: 4.77 ms (started: 2022-04-12 18:46:07 +00:00)


In [20]:
title_emb

tensor([[-0.4357, -0.4224, -0.2424,  ..., -0.3637,  0.1671,  0.4540],
        [-0.3686, -0.6254, -0.7172,  ..., -0.4637,  0.3951, -0.2661],
        [-0.2146, -0.4029,  0.1744,  ..., -0.7837, -0.7946,  0.3658],
        ...,
        [-0.9767, -0.3604, -0.1257,  ..., -0.7109,  0.7241,  0.0203],
        [-0.5571, -0.1584,  0.0423,  ..., -0.0864, -0.2682,  0.5536],
        [ 0.0519, -0.0374,  0.4928,  ..., -0.3209, -0.0911,  0.0488]],
       device='cuda:0')

time: 8.67 ms (started: 2022-04-12 18:46:19 +00:00)


In [ ]:
print(title_emb[results[0][i]['corpus_id']]) 

In [22]:
for i in range(0, len(results[0])):
  print(job_titles[results[0][i]['corpus_id']]) 

Sr. Java Developer
Java Developer
Java Developer
Java Developer
Java-Developer
Junior Java Developer
Application Programmer/Developer (Java)
Sr. Java Developer/Application Systems Engineer 5
Sr. Java Software Engineer
Java Developer (Full-Stack)
time: 1.51 ms (started: 2022-04-12 18:47:38 +00:00)


In [32]:
def job_search(input_text, how_many_results = 2):
    input_embedding = model.encode(input_text, convert_to_tensor=True)
    results = util.semantic_search(input_embedding, title_emb, top_k = how_many_results)
    
    for i in range(0, len(results[0])):
        print(job_titles[results[0][i]['corpus_id']])
        print(job_dict["df"]["job_description"][results[0][i]['corpus_id']])

    


time: 4.85 ms (started: 2022-04-12 18:58:00 +00:00)


In [36]:
job_search("data scientist", 3)

Data Scientist
As a BI/Analytics Developer you will apply your skills and experience in the creation of business intelligence analytics tools on various BI/Analytics platforms(PowerBI, Tableau, Quick sight) for data analysis, dashboarding and reporting of our business data insights. Design and develop data graphs, charts, reports and visualizations that create agent and broker consumable data features. The successful candidate will develop analytic products, metrics and dashboards along the entire development lifecycle, from consulting with end users to capture business needs and processes, designing metrics and tools to optimize presentation, through defining functional requirements and coding, testing, deploying, documenting and maintaining those tools. The BI/Analytics Developer will utilize a full understanding of industry practices, standards and procedures for business intelligence analytics projects. This role will be tasked with resolving a wide range of issues, and will be exp